In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.font_manager as fm
import matplotlib.font_manager as font_manager
from matplotlib.markers import TICKLEFT, TICKRIGHT, TICKUP, TICKDOWN, CARETLEFT, CARETRIGHT, CARETUP, CARETDOWN
from mpl_toolkits.mplot3d import axes3d, Axes3D
import sklearn
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.covariance import GraphicalLassoCV, GraphicalLasso
import scipy
import functools
import seaborn as sns
import glob
import datetime as dt
import gc
import sys
import tqdm as tq
from tqdm import tqdm
import time
import pickle
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import collections

参考文献：https://qiita.com/goodclues/items/0c91f6ac6df6a081fe86

In [6]:
df_all = pd.read_csv('all_sum.csv',encoding='shift-jis')
df_all

,Unnamed: 0,x,d1,d2,d3,d4,d5,w1,w2,w3,...,c3,c4,c5,c6,p,none_s,image_s,announce_s,image_sound_s,image_announce_s
0,0,1,6.2937,1.9398,6.7618,3.07992,5.72156,36.96507,25.87807,21.29791,...,29,21,10,6,3,9.6667,18.9333,22.2000,22.0000,21.0000
1,1,2,11.0338,6.1047,7.2883,7.54295,9.90751,25.40064,16.89495,16.52988,...,14,8,12,21,3,18.0000,18.0000,18.0000,18.0000,18.0000
2,2,3,7.1932,2.1034,4.9684,5.66562,8.38533,29.94836,20.88529,19.59672,...,18,16,18,15,4,14.8000,14.8000,16.4000,15.7333,20.5333
3,3,4,7.8981,-0.9757,4.2858,3.84009,4.87743,26.87720,18.93591,16.00807,...,28,16,17,11,4,13.6667,20.2667,23.4000,19.7333,22.7333
4,4,5,9.7256,0.0349,7.1680,3.90703,6.30194,43.20713,28.51156,24.68393,...,23,24,24,23,5,8.6667,26.4667,27.4667,25.0667,28.1333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,799,800,7.5822,0.1201,6.5722,3.32963,4.75933,22.04394,17.21857,11.89841,...,33,19,18,11,4,12.0000,25.8000,27.0000,27.0000,27.0000
800,800,801,5.6149,-0.7314,1.6795,3.74132,5.60217,29.58012,15.73032,14.89273,...,33,18,24,10,4,14.8000,24.3333,25.6667,22.6667,24.2000
801,801,802,6.0455,2.0958,3.8635,4.97563,5.69270,28.18144,19.74430,19.20815,...,14,16,11,21,2,20.2667,19.4000,21.2000,21.2000,21.2000
802,802,803,8.6738,-0.4216,1.4291,4.27503,5.49286,31.06191,21.50243,19.40287,...,28,26,17,6,4,11.0667,27.6667,25.4667,24.1333,26.0000


In [7]:
df_all = df_all[['d1', 'd2', 'd3', 'd4', 'd5', 'w1', 'w2', 'w3', 'w4',
       'w5', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'p', 'none_s', 'image_s',
       'announce_s', 'image_sound_s', 'image_announce_s']]

In [8]:
new_column_names = {'d1': '安全運転特性', 'd2': '不安定な運転特性','d3': '心配性特性', 'd4': 'ステイタス特性', 'd5': '事前準備特性','w1': '道路状況把握負担', 'w2': '心身状況負担','w3': '運転ペース阻害負担', 'w4': '車環境負担','w5': '視野環境負担','c1': '写真タイプ', 'c2': '三次元映像タイプ','c3': '言語映像タイプ', 'c4': '言語抽象タイプ','c5': '聴覚言語タイプ', 'c6': '聴覚&音タイプ','p': '行動変容フェーズ', 'none_s': '提示なし','image_s': '画像のみ', 'announce_s': '音声のみ','image_sound_s': '画像+報知音', 'image_announce_s': '画像+音声'}

In [13]:
df_all = df_all.rename(columns=new_column_names)
colsname = df_all.columns
colsname

Index(['安全運転特性', '不安定な運転特性', '心配性特性', 'ステイタス特性', '事前準備特性', '道路状況把握負担',
       '心身状況負担', '運転ペース阻害負担', '車環境負担', '視野環境負担', '写真タイプ', '三次元映像タイプ',
       '言語映像タイプ', '言語抽象タイプ', '聴覚言語タイプ', '聴覚&音タイプ', '行動変容フェーズ', '提示なし', '画像のみ',
       '音声のみ', '画像+報知音', '画像+音声'],
      dtype='object')

In [14]:
df_std = df_all.copy()
ss = sklearn.preprocessing.StandardScaler()
df_std = pd.DataFrame(ss.fit_transform(df_std), columns=colsname)
display(df_std)

,安全運転特性,不安定な運転特性,心配性特性,ステイタス特性,事前準備特性,道路状況把握負担,心身状況負担,運転ペース阻害負担,車環境負担,視野環境負担,...,言語映像タイプ,言語抽象タイプ,聴覚言語タイプ,聴覚&音タイプ,行動変容フェーズ,提示なし,画像のみ,音声のみ,画像+報知音,画像+音声
0,0.035945,0.972259,1.408073,-0.567159,0.042771,1.461916,1.633699,1.266865,1.076806,1.018391,...,0.797367,-0.052582,-1.334198,-1.461919,-0.958688,-1.360950,-0.485565,0.108245,0.369601,-0.313742
1,2.116272,3.721150,1.731198,3.555570,3.367288,-0.320694,-0.402525,0.092190,-0.044941,0.010696,...,-1.487863,-2.305007,-0.968901,1.564298,-0.958688,0.839256,-0.733073,-0.981128,-0.635219,-1.108141
2,0.430716,1.080238,0.307426,1.821384,2.158360,0.380319,0.501975,0.847752,0.953193,1.016174,...,-0.878469,-0.918900,0.126991,0.353811,0.089999,-0.005627,-1.581703,-1.396126,-1.204626,-0.437324
3,0.740081,-0.952010,-0.111501,0.135049,-0.627644,-0.093088,0.060104,-0.036366,0.231412,-0.001803,...,0.645019,-0.918900,-0.055658,-0.453180,0.089999,-0.304847,-0.131952,0.419494,-0.199806,0.145236
4,1.542132,-0.285000,1.657367,0.196885,0.503714,2.424105,2.230638,2.101062,2.130905,2.487039,...,-0.116725,0.467208,1.222883,1.967794,1.138687,-1.624976,1.512268,1.474291,1.139971,1.575155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,0.601440,-0.228767,1.291712,-0.336489,-0.721440,-0.838116,-0.329169,-1.048842,-0.447386,-0.479384,...,1.406762,-0.399109,0.126991,-0.453180,0.089999,-0.744899,1.335461,1.353241,1.625626,1.275057
800,-0.261966,-0.790769,-1.711042,0.043810,-0.052049,0.323556,-0.666514,-0.311147,-0.147718,-0.408260,...,1.406762,-0.572373,1.222883,-0.654928,0.089999,-0.005627,0.946497,1.007417,0.537079,0.533618
801,-0.072985,1.075221,-0.370675,1.184005,0.019851,0.107955,0.243344,0.752022,-0.047578,0.199974,...,-1.487863,-0.918900,-1.151550,1.564298,-2.007376,1.437723,-0.361798,-0.151130,0.168637,-0.260782
802,1.080519,-0.586297,-1.864718,0.536825,-0.138864,0.551968,0.641863,0.799994,0.442240,1.047133,...,0.645019,0.813735,-0.055658,-1.461919,0.089999,-0.991314,1.830504,0.955543,0.905497,1.010258


In [17]:
from scipy import stats
import numpy as np
from matplotlib import pyplot as plt

# 正規乱数から得たデータの分布が正規分布と差があるか検定
for i in range(len(df_std.columns)):
    data = df_std.iloc[:,i]
    pv = stats.kstest(data, "norm")[1]
    print(df_std.columns[i],':p-value:' + str(pv))




# # 可視化
# x = np.linspace(-5, 15, 100)
# pdf_norm1 = stats.norm.pdf(x, loc=mu, scale=sigma2)
# pdf_norm2 = stats.norm.pdf(x, loc=mu + 1, scale=sigma2)
# plt.hist(data, normed=True)
# plt.plot(x, pdf_norm1, 'r')
# plt.plot(x, pdf_norm2, 'k')
# plt.show()

安全運転特性 :p-value:0.4675181796306286
不安定な運転特性 :p-value:0.000278040964582732
心配性特性 :p-value:0.0024071523371360574
ステイタス特性 :p-value:0.0010668402916303187
事前準備特性 :p-value:0.02680311795038748
道路状況把握負担 :p-value:0.08162203316855499
心身状況負担 :p-value:0.036145872202899665
運転ペース阻害負担 :p-value:0.06572073585315463
車環境負担 :p-value:0.02003486776049287
視野環境負担 :p-value:0.22345411604200505
写真タイプ :p-value:0.0014167804440525192
三次元映像タイプ :p-value:0.02370521566570205
言語映像タイプ :p-value:0.00814749379391962
言語抽象タイプ :p-value:0.022092757982469003
聴覚言語タイプ :p-value:0.006645247191157667
聴覚&音タイプ :p-value:0.0002486558095671128
行動変容フェーズ :p-value:1.0437092551782778e-32
提示なし :p-value:2.4694496615660342e-05
画像のみ :p-value:4.358556618564098e-05
音声のみ :p-value:0.0003064562789126475
画像+報知音 :p-value:1.4689406250757592e-06
画像+音声 :p-value:0.008775602401715528
